In [1]:
!cd .. && git submodule init
!cd .. && git submodule update
!cd .. && mkdir build
!cd ../build/ && rm -rf *
!cd ../build && cmake \
  -DPYTHON_EXECUTABLE="$(which python)" \
  -DCMAKE_BUILD_TYPE=Release ..
!cd ../build && make install

mkdir: no se puede crear el directorio «build»: El archivo ya existe
-- The C compiler identification is GNU 4.8.4
-- The CXX compiler identification is GNU 4.8.4
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
Release mode
-- Found PythonInterp: /home/alicia/.pyenv/versions/3.6.5/bin/python (found version "3.6.5") 
-- Found PythonLibs: /home/alicia/.pyenv/versions/3.6.5/lib/libpython3.6m.a
-- pybind11 v2.3.dev0
-- Performing Test HAS_FLTO
-- Performing Test HAS_FLTO - Success
-- LTO enabled
CMAKE_INSTALL_PREFIX=/home/alicia/Documentos/Metodos Numericos/TP2/metodos-tp2/metnum-tp2-20191c/metnum-tp2-20191c
-- Configuring done
-- Generating done
-- Build files have been written to: /home/

In [11]:
# Verifico la correcta instalación. Si no falla el import está OK
!pwd
!python --version
import sentiment
from sentiment import entrenar
from sentiment import generate_vocabulary
from sentiment import entrenar_varios

/home/alicia/Documentos/Metodos Numericos/TP2/metodos-tp2/metnum-tp2-20191c/metnum-tp2-20191c/notebooks
Python 3.6.5


In [13]:
%load_ext autoreload
%autoreload 2
import pandas as pd

!cd ../data && tar -xvf *.tgz

df = pd.read_csv("../data/imdb_small.csv", index_col=0)

print("Cantidad de documentos: {}".format(df.shape[0]))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
./._imdb_small.csv
imdb_small.csv
Cantidad de documentos: 12500


In [14]:
text_train = df[df.type == 'train']["review"]
label_train = df[df.type == 'train']["label"]
text_test = df[df.type == 'test']["review"]
label_test = df[df.type == 'test']["label"]

print("Cantidad de instancias de entrenamiento = {}".format(len(text_train)))
print("Cantidad de instancias de test = {}".format(len(text_test)))


Cantidad de instancias de entrenamiento = 6225
Cantidad de instancias de test = 6275


In [15]:
def particion_criticas_equilibradas(text_train, label_train, cant_elems):
    dict_texto = text_train.to_dict()
    dict_label = label_train.to_dict()
    claves_positivos = [x for x in dict_label if dict_label[x] == 'pos']
    claves_negativos = [x for x in dict_label if dict_label[x] == 'neg']
    posiciones_elegidas_pos = random.sample(range(0, len(claves_positivos)), int(cant_elems/2))
    posiciones_elegidas_neg = random.sample(range(0, len(claves_negativos)), int(cant_elems/2))
    nuevo_text_train = {}
    nuevo_label_train = {}
    for i in posiciones_elegidas_pos:
        nuevo_label_train[claves_positivos[i]] = 'pos'
        nuevo_text_train[claves_positivos[i]] = dict_texto[claves_positivos[i]]
    for i in posiciones_elegidas_neg:
        nuevo_label_train[claves_negativos[i]] = 'neg'
        nuevo_text_train[claves_negativos[i]] = dict_texto[claves_negativos[i]]
    return pd.Series(nuevo_text_train), pd.Series(nuevo_label_train)

In [16]:
print("Class balance : {} pos {} neg".format(
    (label_train == 'pos').sum() / label_train.shape[0], 
    (label_train == 'neg').sum() / label_train.shape[0]
))

Class balance : 0.49493975903614457 pos 0.5050602409638554 neg


In [9]:
from sklearn.feature_extraction.text import CountVectorizer
import random

#vectorizer = CountVectorizer(max_df=0.90, min_df=0.01, max_features=5000)
vectorizer = CountVectorizer(stop_words="english", analyzer='word', max_df=0.3, min_df=0.01, max_features=5000)
voc = vectorizer.fit(text_train)

print("Vocabulary content:\n {}".format(voc.vocabulary_))

#criticas_a_experimentar = 1000
#text_train, label_train = particion_criticas_equilibradas(text_train,label_train,criticas_a_experimentar)
#criticas_a_experimentar = 1000
#text_test, label_test = particion_criticas_equilibradas(text_test,label_test,criticas_a_experimentar)


X_train, y_train = vectorizer.transform(text_train), (label_train == 'pos').values
X_test, y_test = vectorizer.transform(text_test), (label_test == 'pos').values

Vocabulary content:
 {'spoiler': 1286, 'read': 1112, 'll': 810, 'know': 755, 'horrible': 666, 'rent': 1141, 'night': 942, 'major': 842, 'results': 1146, 'created': 300, 'waste': 1496, 'building': 182, 'involved': 718, 'leading': 778, 'addition': 39, 'new': 938, 'length': 788, 'necessary': 933, 'different': 364, 'rest': 1144, 'shocking': 1224, 'effects': 417, 'result': 1145, 'right': 1155, 'seriously': 1211, 'probably': 1074, 'worst': 1543, 've': 1466, 'seen': 1201, 'year': 1556, 'low': 835, 'budget': 180, 'acting': 26, 'special': 1282, 'characters': 215, 'clichã': 242, 'act': 24, 'stupid': 1327, 'predictable': 1065, 'ways': 1504, 'walking': 1487, 'dark': 321, 'looking': 819, 'cat': 201, 'falling': 486, 'catch': 202, 'small': 1257, 'room': 1164, 'films': 527, 'takes': 1358, 'possible': 1056, 'cut': 315, 'earlier': 406, 'shots': 1229, 'filming': 524, 'make': 843, 'look': 817, 'reminds': 1140, 'guys': 616, 'working': 1539, 'music': 919, 'children': 226, 'brings': 175, 'aspect': 97, 'compu

Experimento 1. Cambiar cantidad de vecinos k con PCA(10)

In [17]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

def metricas(preds, Y_test):
    tp = np.sum( np.logical_and(preds == 1, Y_test == 1) )
    tn = np.sum( np.logical_and(preds == 0, Y_test == 0) )
    fp = np.sum( np.logical_and(preds == 1, Y_test == 0) )
    fn = np.sum( np.logical_and(preds == 0, Y_test == 1) )
    acc = (tp + tn) / (tp+tn+fp+fn)
    if (tp + fp) == 0:
        prec = 0
    else:
        prec = tp / (tp + fp)
    if (tp + fn) == 0:
        rec = 0
    else:
        rec = tp / (tp + fn)
    return acc, prec, rec

In [23]:
%%time
import sentiment
import numpy as np
import time


alpha = 10

clf = sentiment.PCA(alpha)
clf.fit(X_train.toarray(), 5000, alpha)
X_pda_train = clf.transform(X_train.toarray())
X_pda_test = clf.transform(X_test.toarray())



CPU times: user 4min 25s, sys: 2.58 s, total: 4min 27s
Wall time: 4min 23s


In [25]:
K_vals = np.arange(100,3000,15)
mets = np.zeros((len(K_vals), 4))

for i,k in enumerate(K_vals):
    start_time = time.clock()
    clf2 = sentiment.KNNClassifier(k)
    clf2.fit(X_pda_train, y_train)
    y_pred = clf2.predict(X_pda_test, 1)
    acc, prec, rec = metricas(y_pred, y_test)
    end_time = time.clock() - start_time
    mets[i] = [acc,prec,rec, end_time] 
    print(k)
    

100
115
130
145
160
175
190
205
220
235
250
265
280
295
310
325
340
355
370
385
400
415
430
445
460
475
490
505
520
535
550
565
580
595
610
625
640
655
670
685
700
715
730
745
760
775
790
805
820
835
850
865
880
895
910
925
940
955
970
985
1000
1015
1030
1045
1060
1075
1090
1105
1120
1135
1150
1165
1180
1195
1210
1225
1240
1255
1270
1285
1300
1315
1330
1345
1360
1375
1390
1405
1420
1435
1450
1465
1480
1495
1510
1525
1540
1555
1570
1585
1600
1615
1630
1645
1660
1675
1690
1705
1720
1735
1750
1765
1780
1795
1810
1825
1840
1855
1870
1885
1900
1915
1930
1945
1960
1975
1990
2005
2020
2035
2050
2065
2080
2095
2110
2125
2140
2155
2170
2185
2200
2215
2230
2245
2260
2275
2290
2305
2320
2335
2350
2365
2380
2395
2410
2425
2440
2455
2470
2485
2500
2515
2530
2545
2560
2575
2590
2605
2620
2635
2650
2665
2680
2695
2710
2725
2740
2755
2770
2785
2800
2815
2830
2845
2860
2875
2890
2905
2920
2935
2950
2965
2980
2995


In [26]:
faccuracy = open("variacion_k_alpha10_met2_accuracy", "w")
for i,k in enumerate(K_vals):
    faccuracy.write(" {},".format(float(mets[i,0])))
faccuracy.close()
fprecision = open("variacion_k_alpha10_met2_precision", "w")
for i,k in enumerate(K_vals):
    fprecision.write(" {},".format(float(mets[i,1])))
fprecision.close()
frecall = open("variacion_k_alpha10_met2_recall","w")
for i,k in enumerate(K_vals):
    frecall.write(" {},".format(float(mets[i,2]))) 
frecall.close()
ftime = open("variacion_k_alpha10_met2_time","w")
for i,k in enumerate(K_vals):
    ftime.write(" {},".format(float(mets[i,3])))
ftime.close()


In [ ]:
import matplotlib.pyplot as plt
plt.plot(K_vals,mets[:,0], 'b.-')
plt.plot(K_vals,mets[:,1], 'r.-')
plt.plot(K_vals,mets[:,2], 'g.-')
plt.title("Metricas en funcion de K")
plt.xlabel("K")
plt.ylabel("%")
plt.axis([1, K_vals[-1], 0, 1])
plt.gca().legend(('Acc','Prec','Recall'))
plt.grid()
plt.show()
plt.savefig('variando_k_en_pca10_met1.png')


In [16]:
maxi = []
for i,k in enumerate( K_vals):
    maxi.append(mets[i,0]+mets[i,1]+ mets[i,2])
maximo = np.argmax(maxi)
print(maximo, mets[maximo,:])

126 [0.60446215 0.55965082 0.93154666 2.503035  ]


Experimento 2. Variar alpha con vecinos maximos


In [9]:
%%time
import sentiment
import numpy as np
import time


alpha = 600

clf = sentiment.PCA(alpha)
clf.fit(X_train.toarray(), 1000, alpha)
X_pda_train = clf.transform(X_train.toarray())
X_pda_test = clf.transform(X_test.toarray())

CPU times: user 42min 42s, sys: 23.5 s, total: 43min 5s
Wall time: 42min 20s


In [ ]:
alphas = np.arange(5,600,5)
K_vals_fixes = np.array([750,2000, 2500])

dmets = { i : np.zeros((len(alphas), 4)) for i in (K_vals_fixes)}
dmets

In [21]:


#alphas = np.arange(5,200,5)
#mets = np.zeros((len(alphas), 4))

for i,a in enumerate(alphas):
    start_time = time.clock()
    clf.fit(X_train.toarray(), 1000, a)
    X_pda_train = clf.transform(X_train.toarray())
    X_pda_test = clf.transform(X_test.toarray())
    for j,k in enumerate(K_vals_fixes):
        clf2 = sentiment.KNNClassifier(k)
        clf2.fit(X_pda_train, y_train)
        y_pred = clf2.predict(X_pda_test, 2)
        acc, prec, rec = metricas(y_pred, y_test)
        end_time = time.clock() - start_time        
        mets = dmets[k]
        mets[i] = [acc,prec,rec, end_time] 
    print(a, j)
dmets

5 2
10 2
15 2
20 2
25 2
30 2
35 2
40 2
45 2
50 2
55 2
60 2
65 2
70 2
75 2
80 2
85 2
90 2
95 2
100 2
105 2
110 2
115 2
120 2
125 2
130 2
135 2
140 2
145 2
150 2
155 2
160 2
165 2
170 2
175 2
180 2
185 2
190 2
195 2
200 2
205 2
210 2
215 2
220 2
225 2
230 2
235 2
240 2
245 2
250 2
255 2
260 2
265 2
270 2
275 2
280 2
285 2
290 2
295 2
300 2
305 2
310 2
315 2
320 2
325 2
330 2
335 2
340 2
345 2
350 2
355 2
360 2
365 2
370 2
375 2
380 2
385 2
390 2
395 2
400 2
405 2
410 2
415 2
420 2
425 2
430 2
435 2
440 2
445 2
450 2
455 2
460 2
465 2
470 2
475 2
480 2
485 2
490 2
495 2
500 2
505 2
510 2
515 2
520 2
525 2
530 2
535 2
540 2
545 2
550 2
555 2
560 2
565 2
570 2
575 2
580 2
585 2
590 2
595 2


{750: array([[5.32430279e-01, 6.57032755e-01, 1.10106555e-01, 1.49363600e+00],
        [5.54422311e-01, 6.72000000e-01, 1.89861156e-01, 2.23440300e+00],
        [5.56972112e-01, 6.63233780e-01, 2.07943171e-01, 3.08831600e+00],
        [5.59840637e-01, 6.43776824e-01, 2.42169842e-01, 4.07133200e+00],
        [5.64780876e-01, 6.45933014e-01, 2.61543429e-01, 5.03450700e+00],
        [5.66374502e-01, 6.46417445e-01, 2.68001292e-01, 6.11171500e+00],
        [5.67330677e-01, 6.46248086e-01, 2.72521795e-01, 7.13054500e+00],
        [5.69561753e-01, 6.47761194e-01, 2.80271230e-01, 8.24092000e+00],
        [5.71474104e-01, 6.47612156e-01, 2.88989345e-01, 9.46990400e+00],
        [5.72589641e-01, 6.45003494e-01, 2.98030352e-01, 1.06495200e+01],
        [5.74820717e-01, 6.48033126e-01, 3.03196642e-01, 1.18835000e+01],
        [5.74023904e-01, 6.47426982e-01, 3.00613497e-01, 1.31311420e+01],
        [5.75458167e-01, 6.50871080e-01, 3.01582176e-01, 1.43012760e+01],
        [5.76414343e-01, 6.525364

In [22]:
for j,k in enumerate(K_vals_fixes):
    mets = dmets[k]    
    faccuracy = open("variacion_alpha_hasta600_{0}_met1_accuracy".format(k), "w")
    for i,l in enumerate(alphas):
        faccuracy.write(" {},".format(float(mets[i,0])))
    faccuracy.close()
    fprecision = open("variacion_alpha_hasta600_{0}_met1_precision".format(k), "w")
    for i,l in enumerate(alphas):
        fprecision.write(" {},".format(float(mets[i,1])))
    fprecision.close()
    frecall = open("variacion_alpha_hasta600_{0}_met1_recall".format(k),"w")
    for i,l in enumerate(alphas):
        frecall.write(" {},".format(float(mets[i,2]))) 
    frecall.close()
    ftime = open("variacion_alpha_hasta600_{0}_met1_time".format(k),"w")
    for i,l in enumerate(alphas):
        ftime.write(" {},".format(float(mets[i,3])))
    ftime.close()

In [ ]:
import matplotlib.pyplot as plt
line_styles = ['b.-', 'r.-', 'g.-', 'k.-', 'y.-', 'm.-']


mets = dmets[750]
plt.subplot(3, 1, 1)
plt.plot(alphas,mets[:,0], 'b.--')
plt.plot(alphas,mets[:,1], 'r.--')
plt.plot(alphas,mets[:,2], 'g.--')
plt.ylabel("k =750 ")
plt.grid()
plt.axis([1, alphas[-1], 0, 0.8])
plt.title("Metricas KNN+PCA variando alpha para distintos k fijos metodo 1 ")   
mets = dmets[2000]
plt.subplot(3, 1, 2)
plt.plot(alphas,mets[:,0], 'b.--')
plt.plot(alphas,mets[:,1], 'r.--')
plt.plot(alphas,mets[:,2], 'g.--')
plt.ylabel("k =2000 ")
plt.grid()
plt.axis([1, alphas[-1], 0, 0.8])
mets = dmets[2500]
plt.subplot(3, 1, 3)
plt.plot(alphas,mets[:,0], 'b.--')
plt.plot(alphas,mets[:,1], 'r.--')
plt.plot(alphas,mets[:,2], 'g.--')
plt.ylabel("k=2500")
plt.grid()         
plt.xlabel("alpha")
plt.axis([1, alphas[-1], 0, 0.8])
plt.gca().legend(('Acc','Prec','Recall'), loc="upper left", bbox_to_anchor = (0.9, 0.3))
plt.savefig('variando_alpha_kinterestantes_met1_pormetrica.png')

In [71]:
maximos = []
max_metrics = []
for j, k in enumerate(K_vals_fixes):
    maxi = []
    mets = dmets[k]
    for i,l in enumerate( alphas):
        maxi.append(mets[i,0]+mets[i,1]+ mets[i,2])
    maximo = np.argmax(maxi)
    maximos.append(maximo)
    max_metrics.append(mets[maximo,:])
print(maximos, max_metrics)

[47, 26, 4] [array([ 0.57800797,  0.64897147,  0.31578947, 63.894359  ]), array([ 0.53912351,  0.56847027,  0.27478205, 65.097472  ]), array([ 0.51266932,  0.72413793,  0.02034227, 16.252652  ])]


In [31]:
import matplotlib.pyplot as plt
import numpy as np
import csv

def levantar_archivo(nombre):
    f = open(nombre, "r")
    aux = (f.readline()).split()
    aux = [float(i) for i in aux]
    return aux

In [ ]:
K_vals = np.arange(100,3000,15)
accuracy = levantar_archivo('variacion_k_alpha10_met1_accuracy')
precision = levantar_archivo('variacion_k_alpha10_met1_precision')
recall = levantar_archivo('variacion_k_alpha10_met1_recall')
plt.plot(K_vals,accuracy, 'b.-')
plt.plot(K_vals,precision, 'r.-')
plt.plot(K_vals,recall, 'g.-')
plt.title("Metricas en funcion de K")
plt.xlabel("alpha")
plt.ylabel("Porcentaje")
plt.axis([1,K_vals[-1], 0, 1])
plt.grid()
plt.gca().legend(('Acc','Prec','Recall'), loc="best", bbox_to_anchor = (0.9, 0.3))
plt.savefig('variacion_k_alpha10_met1.png')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
alphas = np.arange(5,600,5)
line_styles = ['b.-', 'r.-', 'g.-', 'k.-', 'y.-', 'm.-']


metacc =  levantar_archivo('variacion_alpha_hasta600_86_met1_accuracy')
metprec = levantar_archivo('variacion_alpha_hasta600_86_met1_precision')
metrecall = levantar_archivo('variacion_alpha_hasta600_86_met1_recall')

plt.subplot(3, 1, 1)
plt.plot(alphas,metacc, 'b.--')
plt.plot(alphas,metprec, 'r.--')
plt.plot(alphas,metrecall, 'g.--')
plt.ylabel("k =86 ")
plt.grid() 
plt.title("Metricas en funcion de alpha con k fijos metodo 1")    
metacc =  levantar_archivo('variacion_alpha_hasta600_1425_met1_accuracy')
metprec = levantar_archivo('variacion_alpha_hasta600_1425_met1_precision')
metrecall = levantar_archivo('variacion_alpha_hasta600_1425_met1_recall')
plt.subplot(3, 1, 2)
plt.plot(alphas,metacc, 'b.--')
plt.plot(alphas,metprec, 'r.--')
plt.plot(alphas,metrecall, 'g.--')
plt.ylabel("k =1450 ")
plt.grid()     
metacc =  levantar_archivo('variacion_alpha_hasta600_2500_met1_accuracy')
metprec = levantar_archivo('variacion_alpha_hasta600_2500_met1_precision')
metrecall = levantar_archivo('variacion_alpha_hasta600_2500_met1_recall')
plt.subplot(3, 1, 3)
plt.plot(alphas,metacc, 'b.--')
plt.plot(alphas,metprec, 'r.--')
plt.plot(alphas,metrecall, 'g.--')
plt.ylabel("k=2500")        
plt.xlabel("alpha")

    #plt.xticks(alphas)
plt.axis([1, alphas[-1], 0, 1])
plt.grid()         
plt.gca().legend(('Acc','Prec','Recall'), loc="upper left", bbox_to_anchor = (0.9, 0.3))
plt.savefig('variando_alpha_kinterestantes600_met1_pormetrica.png')

In [7]:
def particion_criticas_equilibradas(text_train, label_train, cant_elems):
    dict_texto = text_train.to_dict()
    dict_label = label_train.to_dict()
    claves_positivos = [x for x in dict_label if dict_label[x] == 'pos']
    claves_negativos = [x for x in dict_label if dict_label[x] == 'neg']
    posiciones_elegidas_pos = random.sample(range(0, len(claves_positivos)), int(cant_elems/2))
    posiciones_elegidas_neg = random.sample(range(0, len(claves_negativos)), int(cant_elems/2))
    nuevo_text_train = {}
    nuevo_label_train = {}
    for i in posiciones_elegidas_pos:
        nuevo_label_train[claves_positivos[i]] = 'pos'
        nuevo_text_train[claves_positivos[i]] = dict_texto[claves_positivos[i]]
    for i in posiciones_elegidas_neg:
        nuevo_label_train[claves_negativos[i]] = 'neg'
        nuevo_text_train[claves_negativos[i]] = dict_texto[claves_negativos[i]]
    return pd.Series(nuevo_text_train), pd.Series(nuevo_label_train)

In [8]:
print("Class balance : {} pos {} neg".format(
    (label_train == 'pos').sum() / label_train.shape[0], 
    (label_train == 'neg').sum() / label_train.shape[0]
))

Class balance : 0.49493975903614457 pos 0.5050602409638554 neg


In [ ]:
Tiempos knn y PCA+KNN

In [47]:
alpha = [10,100,200,300]
mets = np.zeros((len(K_vals),1))

clf = sentiment.PCA(a)
clf.fit(X_train.toarray(), 1000, 300)

for i, a in enumerate(alpha):
    start_time = time.clock()
    clf.fit(X_train.toarray(), 1000, a)
    X_pda_train = clf.transform(X_train.toarray())
    X_pda_test = clf.transform(X_test.toarray())
    end_time = time.clock() - start_time
    mets[i] = [end_time]


In [48]:
ftime = open("tiempos_conpca_overhead_ayudado_met0_time","w")
for i,k in enumerate(K_vals):
    ftime.write(" {}".format(float(mets[i,0])))
ftime.close()

In [58]:
def f1score(precision, recall):
    return 2*((precision * recall) /(precision + recall))

In [93]:
precision = levantar_archivo('variacion_alpha_hasta600_2500_met1_precision')
recall = levantar_archivo('variacion_alpha_hasta600_2500_met1_recall')

fscore = np.zeros(len(precision))
for i in range(0,len(precision)):
    if((precision[i] == 0) and (recall[i]==0)):
        fscore[i] = 0
    else:
        fscore[i] = (f1score(precision[i], recall[i]))
                      


In [94]:
filescore = open("variacion_alpha_hasta600_2500_met1_f1score","w")
for i,k in enumerate(fscore):
    filescore.write(" {}".format(float(fscore[i])))
filescore.close()

In [ ]:
%%time
from sklearn.feature_extraction.text import CountVectorizer
import random
import sentiment
import numpy as np
import time


#SIn PCA
samples_size = [6000, 5000, 4000, 3000, 2000, 1000]
#K= 100, alpha = 10, alpha = 100
iterations = 25
times = np.zeros((len(samples_size),1))
for j, k in enumerate(samples_size):
    criticas_a_experimentar = k
    text_train, label_train = particion_criticas_equilibradas(text_train,label_train,criticas_a_experimentar)
    vectorizer = CountVectorizer(stop_words="english", analyzer='word', max_df=0.3, min_df=0.01, max_features=5000)
    voc = vectorizer.fit(text_train)    
    X_train, y_train = vectorizer.transform(text_train), (label_train == 'pos').values
    X_test, y_test = vectorizer.transform(text_test), (label_test == 'pos').values
    mets = []
    for i in range(0,iterations):
        start_time = time.clock()     
        clf = sentiment.KNNClassifier(100)
        clf.fit(X_train.toarray(), y_train)
        y_pred = clf.predict(X_test.toarray(), 0)
        end_time = time.clock() - start_time    
        mets.append(end_time)
        print(i)
    times[j] = (sum(mets)/len(mets))
    print(j)


In [ ]:
%%time
from sklearn.feature_extraction.text import CountVectorizer
import random
import sentiment
import numpy as np
import time


#SIn PCA
samples_size = [6000, 5000, 4000, 3000, 2000, 1000]
#K= 100, alpha = 10, alpha = 100
iterations = 25
alpha= 100
times = np.zeros((len(samples_size),1))
for j, k in enumerate(samples_size):
    criticas_a_experimentar = k
    text_train, label_train = particion_criticas_equilibradas(text_train,label_train,criticas_a_experimentar)
    vectorizer = CountVectorizer(stop_words="english", analyzer='word', max_df=0.3, min_df=0.01, max_features=5000)
    voc = vectorizer.fit(text_train)    
    X_train, y_train = vectorizer.transform(text_train), (label_train == 'pos').values
    X_test, y_test = vectorizer.transform(text_test), (label_test == 'pos').values
    mets = []
    clf = sentiment.PCA(alpha)
    clf.fit(X_train.toarray(), 1000, alpha)
    X_pda_train = clf.transform(X_train.toarray())
    X_pda_test = clf.transform(X_test.toarray())   
    for i in range(0,iterations):
        start_time = time.clock()     
        clf = sentiment.KNNClassifier(100)
        clf.fit(X_pda_train, y_train)
        y_pred = clf.predict(X_pda_test, 0)
        end_time = time.clock() - start_time    
        mets.append(end_time)
        print(i)
    times[j] = (sum(mets)/len(mets))
    print(j)


In [19]:
filetimes = open("tiempos_conpca100_variando_tamaño","w")
for i,k in enumerate(times):
    filetimes.write(" {}".format(float(times[i])))
filetimes.close()